Proyek ini bertujuan untuk mengklasifikasikan sentimen (0: Negatif, 1: Positif) dari 10.000 ulasan aplikasi MyIM3. Secara kritis, proyek ini menguji hipotesis bahwa model Transformer (IndoBERT) yang sudah pre-trained akan mengungguli model sekuensial tradisional (LSTM), baik yang dilatih from scratch (dari nol) maupun yang diperkuat dengan pre-trained word embedding (FastText).


In [1]:
!pip install transformers datasets scikit-learn torch
from sklearn.model_selection import train_test_split
!pip install kagglehub[pandas-datasets]

Import Dataset & Praproses

In [2]:
import pandas as pd
import re
import kagglehub
from kagglehub import KaggleDatasetAdapter

# 1️⃣ Unduh dataset langsung dari Kaggle
# (file_path diisi dengan nama file di dalam dataset)
file_path = "myIM3_Beli_Pulsa__Cek_Kuota.csv"

df = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "ucupsedaya/10k-myim3-app-reviews",
    file_path
)

print("✅ Dataset berhasil dimuat dari Kaggle!")
print(df.head())

# 2️⃣ Rename kolom agar konsisten
df = df.rename(columns={'content': 'Text Review', 'score': 'Rating'}, errors='ignore')

# 3️⃣ Fungsi pembersihan teks
def clean_text(text):
    text = re.sub(r'#(\w+)', r'\1', str(text))
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'RT\s', '', text)
    text = re.sub(r'([.!?])\1+', r'\1', text)
    text = re.sub(r'[\n\t]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# 4️⃣ Bersihkan teks
df['cleaned_text'] = df['Text Review'].apply(clean_text)

# 5️⃣ Coba tidak hapus yang netral | Konversi rating ke label biner (hapus yang netral)
def rating_to_binary_label(rating):
    if rating >= 3:
        return 1
    elif rating <= 2:
        return 0
    return None

df['label'] = df['Rating'].apply(rating_to_binary_label)
df.dropna(subset=['label'], inplace=True)
df['label'] = df['label'].astype(int)

# 6️⃣ Pilih kolom akhir
df = df.rename(columns={'Text Review': 'Text Tweet'}, errors='ignore')
df = df[['cleaned_text', 'label']]

# 7️⃣ Cek hasil akhir
print(f"✅ Data bersih siap: {len(df)} baris.")
print("Distribusi label:\n", df['label'].value_counts())

100%|██████████| 411k/411k [00:00<00:00, 578kB/s]

Extracting zip of myIM3_Beli_Pulsa__Cek_Kuota.csv...
✅ Dataset berhasil dimuat dari Kaggle!
                userName                                            content  \
0   UCIL OLOL LEHO [UOL]  Kenapa Indosat ada tuyulnya sekarang pulsa tib...   
1              Pp.joaana                          aplikasi ini sangat bagus   
2              Xeraphine  Tolong itu sistem login nomor utama dan nomor ...   
3  Sugeng dian pamungkas  Sekarang pilihanya yg 30 hari naik terus harga...   
4             Bang Tinus                                              murah   

   score                   at appVersion  
0      1  2024-02-02 05:17:07     82.0.5  
1      5  2024-02-02 05:16:51     82.0.5  
2      3  2024-02-02 05:13:29        NaN  
3      4  2024-02-02 05:13:07        NaN  
4      5  2024-02-02 05:11:20    81.16.0  
✅ Data bersih siap: 10000 baris.
Distribusi label:
 label
0    5123
1    4877
Name: count, dtype: int64


Split Dataset

In [3]:
# Cell 4: Split Dataset
# Pisahkan 20% data untuk Test Set
df_train_val, df_test = train_test_split(df,
                                         test_size=0.2,
                                         random_state=42,
                                         stratify=df['label'])

# Bagi sisa data menjadi Train dan Validation
df_train, df_val = train_test_split(df_train_val,
                                    test_size=0.2,
                                    random_state=42,
                                    stratify=df_train_val['label'])

print(f"Jumlah data training: {len(df_train)}")
print(f"Jumlah data validasi: {len(df_val)}")
print(f"Jumlah data test: {len(df_test)}")

Jumlah data training: 6400
Jumlah data validasi: 1600
Jumlah data test: 2000


# Indobert

Tokenisasi

In [ ]:
# Cell 5: Tokenisasi (Transformer)
from transformers import AutoTokenizer

NAMA_MODEL = "indobenchmark/indobert-base-p2"
tokenizer = AutoTokenizer.from_pretrained(NAMA_MODEL)
MAX_LEN = 128

train_encodings = tokenizer(df_train['cleaned_text'].tolist(), truncation=True, padding=True, max_length=MAX_LEN)
val_encodings = tokenizer(df_val['cleaned_text'].tolist(), truncation=True, padding=True, max_length=MAX_LEN)
test_encodings_trans = tokenizer(df_test['cleaned_text'].tolist(), truncation=True, padding=True, max_length=MAX_LEN)

print("✅ Tokenisasi Transformer selesai.")

Membuat Dataset (PyTorch)

In [5]:
# Cell 6: Dataset PyTorch & Metrik
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Definisikan Class Dataset
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Buat dataset untuk Transformer
train_dataset = SentimentDataset(train_encodings, df_train['label'].tolist())
val_dataset = SentimentDataset(val_encodings, df_val['label'].tolist())
test_dataset_trans = SentimentDataset(test_encodings_trans, df_test['label'].tolist())

# Definisikan fungsi untuk menghitung metrik
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = precision_score(labels, predictions, average='macro', zero_division=0)
    recall = recall_score(labels, predictions, average='macro', zero_division=0)
    f1 = f1_score(labels, predictions, average='macro', zero_division=0)
    acc = accuracy_score(labels, predictions)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

print("✅ Dataset dan Fungsi Metrik siap.")

✅ Dataset dan Fungsi Metrik siap.


In [6]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Definisikan fungsi untuk menghitung metrik
def compute_metrics(eval_pred):
    # 'eval_pred' adalah tuple berisi (predictions, labels)

    # 1. Ambil logits dan label aslinya
    logits, labels = eval_pred

    # 2. Ubah logits menjadi tebakan (0 atau 1)
    predictions = np.argmax(logits, axis=-1)

    # 3. Hitung metrik menggunakan sklearn (macro average)
    precision = precision_score(labels, predictions, average='macro')
    recall = recall_score(labels, predictions, average='macro')
    f1 = f1_score(labels, predictions, average='macro')
    acc = accuracy_score(labels, predictions)

    # 4. Kembalikan dalam bentuk dictionary
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

print("✅ Fungsi compute_metrics siap.")

✅ Fungsi compute_metrics siap.


Training Model (Fine-Tuning) - Transformer

In [8]:
# Cell 7: Training Model Transformer
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
# ✨ TAMBAHAN: Import library ini untuk mematikan bar download
from huggingface_hub import utils

# ✨ TAMBAHAN: Matikan progress bar download
utils.disable_progress_bars()

NAMA_MODEL = "indobenchmark/indobert-base-p2"

# Bar download (hijau) tidak akan muncul lagi setelah baris di atas
model_trans = AutoModelForSequenceClassification.from_pretrained(NAMA_MODEL, num_labels=2)

training_args = TrainingArguments(
    output_dir='./results_myim3_trans',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs_myim3_trans',
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
    disable_tqdm=True # Ini mematikan bar saat training (looping)
)

trainer_trans = Trainer(
    model=model_trans,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

print("Memulai Training Transformer (IndoBERT)...")
trainer_trans.train()
print("✅ Training Transformer selesai.")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Memulai Training Transformer (IndoBERT)...
{'loss': 0.5934, 'grad_norm': 4.419984817504883, 'learning_rate': 4.9000000000000005e-06, 'epoch': 0.125}
{'loss': 0.3426, 'grad_norm': 2.623445987701416, 'learning_rate': 9.900000000000002e-06, 'epoch': 0.25}
{'loss': 0.3513, 'grad_norm': 2.0583078861236572, 'learning_rate': 1.49e-05, 'epoch': 0.375}
{'loss': 0.3353, 'grad_norm': 4.750554084777832, 'learning_rate': 1.9900000000000003e-05, 'epoch': 0.5}
{'loss': 0.3315, 'grad_norm': 3.976949691772461, 'learning_rate': 2.4900000000000002e-05, 'epoch': 0.625}
{'loss': 0.3052, 'grad_norm': 2.7730374336242676, 'learning_rate': 2.9900000000000002e-05, 'epoch': 0.75}
{'loss': 0.3456, 'grad_norm': 2.2314631938934326, 'learning_rate': 3.49e-05, 'epoch': 0.875}
{'loss': 0.3332, 'grad_norm': 19.415319442749023, 'learning_rate': 3.99e-05, 'epoch': 1.0}
{'eval_loss': 0.3543838560581207, 'eval_accuracy': 0.895, 'eval_f1': 0.8947234846526579, 'eval_precision': 0.8964122007809405, 'eval_recall': 0.8942776735

Evaluasi Model - Transformer

In [9]:
# Cell 8: Evaluasi Model Transformer
print("Memulai evaluasi final Transformer pada Test Set...")
test_results_trans = trainer_trans.evaluate(eval_dataset=test_dataset_trans)

print("\n--- Hasil Evaluasi Test Set Transformer (IndoBERT) ---")
print(test_results_trans)

Memulai evaluasi final Transformer pada Test Set...
{'eval_loss': 0.335831880569458, 'eval_accuracy': 0.8845, 'eval_f1': 0.8844560644184951, 'eval_precision': 0.8844003424671236, 'eval_recall': 0.8845653533458411, 'eval_runtime': 12.5841, 'eval_samples_per_second': 158.93, 'eval_steps_per_second': 2.543, 'epoch': 3.0}

--- Hasil Evaluasi Test Set Transformer (IndoBERT) ---
{'eval_loss': 0.335831880569458, 'eval_accuracy': 0.8845, 'eval_f1': 0.8844560644184951, 'eval_precision': 0.8844003424671236, 'eval_recall': 0.8845653533458411, 'eval_runtime': 12.5841, 'eval_samples_per_second': 158.93, 'eval_steps_per_second': 2.543, 'epoch': 3.0}


In [10]:
import numpy as np
from sklearn.metrics import classification_report

# 1. Dapatkan prediksi mentah (logits) dari model Transformer (IndoBERT)
print("Memulai prediksi dengan model Transformer (IndoBERT) pada Test Set...")
pred_output = trainer_trans.predict(test_dataset_trans)

# 2. Ambil logits dan label aslinya
logits = pred_output.predictions
y_true_bert = pred_output.label_ids

# 3. Ubah logits menjadi label (0 atau 1), sama seperti di compute_metrics
y_pred_bert = np.argmax(logits, axis=1)

# 4. Tampilkan classification_report
print("\n--- Hasil Evaluasi Test Set Transformer (IndoBERT) [Format Laporan] ---")
print(classification_report(
    y_true_bert,
    y_pred_bert,
    target_names=['Negative (0)', 'Positive (1)'], # Pastikan urutannya benar
    zero_division=0
))

Memulai prediksi dengan model Transformer (IndoBERT) pada Test Set...

--- Hasil Evaluasi Test Set Transformer (IndoBERT) [Format Laporan] ---
              precision    recall  f1-score   support

Negative (0)       0.89      0.88      0.89      1025
Positive (1)       0.88      0.89      0.88       975

    accuracy                           0.88      2000
   macro avg       0.88      0.88      0.88      2000
weighted avg       0.88      0.88      0.88      2000



# LSTM

Tokenisasi & Sequence LSTM

In [ ]:
# Cell 9 Tokenisasi & Sequence (LSTM)
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Parameter untuk Tokenizer LSTM
LSTM_MAX_LEN = 128
MAX_WORDS = 20000

# --- INI ADALAH PERBAIKANNYA ---
# Kita definisikan filter default Keras, tapi KITA HAPUS "0123456789" dari daftarnya
# sehingga angka tetap aman.
custom_filters = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'

# 1. Inisialisasi tokenizer DENGAN FILTER BARU
tokenizer_lstm = Tokenizer(
    num_words=MAX_WORDS,
    oov_token="<oov>",
    filters=custom_filters, # <-- Tambahkan baris ini
    lower=True # <-- Pastikan ini True (memang default-nya)
)

# 2. Inisialisasi dan fit tokenizer
tokenizer_lstm.fit_on_texts(df_train['cleaned_text'].tolist())

# 3. Ubah teks menjadi sequence integer
train_sequences = tokenizer_lstm.texts_to_sequences(df_train['cleaned_text'].tolist())
val_sequences = tokenizer_lstm.texts_to_sequences(df_val['cleaned_text'].tolist())
test_sequences = tokenizer_lstm.texts_to_sequences(df_test['cleaned_text'].tolist())

# 4. Padding sequences
X_train_lstm = pad_sequences(train_sequences, maxlen=LSTM_MAX_LEN, padding='post', truncating='post')
X_val_lstm = pad_sequences(val_sequences, maxlen=LSTM_MAX_LEN, padding='post', truncating='post')
X_test_lstm = pad_sequences(test_sequences, maxlen=LSTM_MAX_LEN, padding='post', truncating='post')

# 5. Ambil label
y_train_lstm = np.array(df_train['label'].tolist())
y_val_lstm = np.array(df_val['label'].tolist())
y_test_lstm = np.array(df_test['label'].tolist())

print("✅ Tokenisasi dan Padding LSTM (dengan filter angka) selesai.")
print(f"Contoh kata di kamus: 'myim3' ada di indeks {tokenizer_lstm.word_index.get('myim3')}")

✅ Tokenisasi dan Padding LSTM (dengan filter angka) selesai.
Contoh kata di kamus: 'myim3' ada di indeks 110


Model LSTM & Training

In [ ]:
# Cell 10: Model LSTM dan Training
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2 # Import l2 regularizer

# Model Hyperparameters
EMBEDDING_DIM = 128
VOCAB_SIZE = MAX_WORDS
LSTM_UNITS = 16 # Sesuaikan dengan model sederhana

# 1. Definisikan Model LSTM (Sudah disederhanakan)
model_lstm = Sequential([ # Rename model_lstm_simple to model_lstm
    Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=LSTM_MAX_LEN),
    Bidirectional(LSTM(LSTM_UNITS, # Gunakan LSTM_UNITS yang baru
                       kernel_regularizer=l2(0.01))), # Cukup satu layer Bi-LSTM
    Dropout(0.5),
    Dense(1, activation='sigmoid') # Langsung ke output
])

# Definisikan callback-nya
early_stop = EarlyStopping(monitor='val_loss', # Pantau loss validasi
                           patience=2,       # Berhenti jika 2 epoch tidak membaik
                           restore_best_weights=True) # Kembalikan ke bobot terbaik

# 2. Compile Model (Gunakan NAMA YANG BENAR)
model_lstm.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model_lstm.summary() # Tampilkan summary dari model yang benar

# 3. Training Model (Gunakan NAMA YANG BENAR)
print("\nMemulai Training Model LSTM...")
history_lstm = model_lstm.fit( # Gunakan model_lstm
    X_train_lstm, y_train_lstm,
    epochs=5,
    batch_size=32,
    validation_data=(X_val_lstm, y_val_lstm),
    verbose=1,
    callbacks=[early_stop]
)

print("✅ Training LSTM selesai.")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_23 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_20                │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_42 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


Memulai Training Model LSTM...
Epoch 1/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 12s 37ms/step - accuracy: 0.7199 - loss: 1.4664 - val_accuracy: 0.8325 - val_loss: 0.4538
Epoch 2/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.8496 - loss: 0.4304 - val_accuracy: 0.8675 - val_loss: 0.3665
Epoch 3/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.8900 - loss: 0.3466 - val_accuracy: 0.8737 - val_loss: 0.3492
Epoch 4/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.9124 - loss: 0.2999 - val_accuracy: 0.8750 - val_loss: 0.3670
Epoch 5/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.9248 - loss: 0.2642 - val_accuracy: 0.8838 - val_loss: 0.3773
✅ Training LSTM selesai.


Evaluasi Model

In [ ]:
# Cell 11: Evaluasi Model LSTM
from sklearn.metrics import classification_report

print("Memulai evaluasi final LSTM pada Test Set...")

# 1. Prediksi Probability
y_pred_proba_lstm = model_lstm.predict(X_test_lstm, verbose=0)

# 2. Konversi ke Label Biner (0 atau 1)
y_pred_lstm = (y_pred_proba_lstm > 0.5).astype(int)

# 3. Hitung Metrik
report_lstm = classification_report(y_test_lstm, y_pred_lstm,
                                    target_names=['Negative (0)', 'Positive (1)'],
                                    output_dict=True, zero_division=0)

print("\n--- Hasil Evaluasi Test Set LSTM ---")
print(classification_report(y_test_lstm, y_pred_lstm,
                            target_names=['Negative (0)', 'Positive (1)'],
                            zero_division=0))

# Ekstrak F1 Macro dan Accuracy
acc_lstm = report_lstm['accuracy']
f1_lstm = report_lstm['macro avg']['f1-score']

Memulai evaluasi final LSTM pada Test Set...

--- Hasil Evaluasi Test Set LSTM ---
              precision    recall  f1-score   support

Negative (0)       0.88      0.87      0.87      1025
Positive (1)       0.87      0.87      0.87       975

    accuracy                           0.87      2000
   macro avg       0.87      0.87      0.87      2000
weighted avg       0.87      0.87      0.87      2000



# LSTM + FastText(cc.id)

Unduh Pre-trained Embedding (FastText cc.id)

In [ ]:
# Cell 11.A: Unduh Pre-trained Embedding (FastText cc.id)
import requests
import gzip
import shutil
import os
from tqdm.notebook import tqdm

# URL dan nama file
url = "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.vec.gz"
gz_filename = "cc.id.300.vec.gz"
vec_filename = "cc.id.300.vec"

# Fungsi untuk menampilkan progress bar
def download_with_progress(url, filename):
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        total_size = int(r.headers.get('content-length', 0))
        block_size = 8192
        with open(filename, 'wb') as f:
            with tqdm(total=total_size, unit='iB', unit_scale=True, desc=filename) as pbar:
                for chunk in r.iter_content(chunk_size=block_size):
                    pbar.update(len(chunk))
                    f.write(chunk)

try:
    if os.path.exists(vec_filename):
        print(f"✅ File '{vec_filename}' sudah ada. Tidak perlu mengunduh lagi.")
    else:
        # 1. Mengunduh file
        print(f"📥 Mengunduh file dari {url}...")
        if not os.path.exists(gz_filename):
            download_with_progress(url, gz_filename)

        # 2. Mengekstrak file .gz
        print(f"⚙️ Mengekstrak '{gz_filename}'... (Ini mungkin butuh beberapa menit)")
        with gzip.open(gz_filename, 'rb') as f_in:
            with open(vec_filename, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print(f"✅ Ekstraksi selesai. File '{vec_filename}' telah dibuat.")

        # 3. Hapus file .gz
        os.remove(gz_filename)
        print(f"🗑️ File '{gz_filename}' telah dihapus untuk menghemat ruang.")

except Exception as e:
    print(f"❌ Terjadi kesalahan: {e}")

Embedding Matrix

In [ ]:
# Cell 11.B: Siapkan Embedding Matrix

print("📖 Memuat pre-trained word vectors dari cc.id.300.vec...")
embeddings_index = {}
try:
    with open(vec_filename, 'r', encoding='utf-8') as f:
        # Lewati baris header (jika ada, file cc.id biasanya ada)
        next(f)
        for line in f:
            values = line.split()
            word = values[0]
            try:
                coefs = np.asarray(values[1:], dtype='float32')
                embeddings_index[word] = coefs
            except ValueError:
                pass # Abaikan baris yang bermasalah

except FileNotFoundError:
    print(f"❌ ERROR: File '{vec_filename}' tidak ditemukan. Jalankan Cell 11.A dulu.")
    # Hentikan eksekusi jika file tidak ada
    raise

print(f'✅ Ditemukan {len(embeddings_index)} word vectors.')

# --- Siapkan Matrix ---
# Gunakan variabel dari Cell 9 & 10
EMBEDDING_DIM_FT = 300 # Ini harus 300 sesuai nama file
# Pastikan VOCAB_SIZE sama dengan yang di-set di Cell 10 (yaitu MAX_WORDS)
VOCAB_SIZE = MAX_WORDS

embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM_FT))

# tokenizer_lstm diambil dari Cell 9
for word, i in tokenizer_lstm.word_index.items():
    if i >= VOCAB_SIZE:
        continue # Hanya proses kata dalam batas MAX_WORDS

    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Kata ditemukan di FastText, masukkan ke matrix
        embedding_matrix[i] = embedding_vector

print("✅ Embedding matrix siap.")

📖 Memuat pre-trained word vectors dari cc.id.300.vec...
✅ Ditemukan 2000000 word vectors.
✅ Embedding matrix siap.


Model LSTM + FastText & Training

In [ ]:
# Cell 11.C: Model LSTM + FastText & Training

from tensorflow.keras.callbacks import EarlyStopping # Import EarlyStopping

# Define EarlyStopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True) # Define early_stop

# 1. Definisikan Model LSTM + FastText
model_lstm_ft = Sequential([
    # Layer Embedding (Menggunakan FastText, tidak dilatih)
    Embedding(
        VOCAB_SIZE,
        EMBEDDING_DIM_FT,
        weights=[embedding_matrix], # <-- INI BAGIAN PENTING
        input_length=LSTM_MAX_LEN,
        trainable=False # <-- INI JUGA PENTING
    ),

    LSTM(128, return_sequences=True),
    Dropout(0.3),
    Bidirectional(LSTM(64)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# 2. Compile Model
model_lstm_ft.compile(
    optimizer=Adam(learning_rate=0.0003),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model_lstm_ft.summary()

# 3. Training Model
print("\nMemulai Training Model LSTM + FastText...")
history_lstm_ft = model_lstm_ft.fit(
    X_train_lstm, y_train_lstm, # Data dari Cell 9
    epochs=5, # Kita samakan 5 epoch untuk perbandingan
    batch_size=32,
    validation_data=(X_val_lstm, y_val_lstm), # Data dari Cell 9
    verbose=1,
    callbacks=[early_stop]
)

print("✅ Training LSTM + FastText selesai.")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_24 (Embedding)        │ ?                      │     6,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_32 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_43 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_21                │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_44 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_45 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,000,000 (22.89 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 6,000,000 (22.89 MB)


Memulai Training Model LSTM + FastText...
Epoch 1/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.6950 - loss: 0.5759 - val_accuracy: 0.8594 - val_loss: 0.3662
Epoch 2/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.8672 - loss: 0.3848 - val_accuracy: 0.8631 - val_loss: 0.3535
Epoch 3/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.8680 - loss: 0.3689 - val_accuracy: 0.8650 - val_loss: 0.3539
Epoch 4/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.8703 - loss: 0.3694 - val_accuracy: 0.8644 - val_loss: 0.3484
Epoch 5/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.8743 - loss: 0.3585 - val_accuracy: 0.8687 - val_loss: 0.3467
✅ Training LSTM + FastText selesai.


Evaluasi Model LSTM + FastText

In [ ]:
# Cell 11.D: Evaluasi Model LSTM + FastText

print("Memulai evaluasi final LSTM + FastText pada Test Set...")

# 1. Prediksi Probability
y_pred_proba_lstm_ft = model_lstm_ft.predict(X_test_lstm, verbose=0) # Data dari Cell 9

# 2. Konversi ke Label Biner (0 atau 1)
y_pred_lstm_ft = (y_pred_proba_lstm_ft > 0.5).astype(int)

# 3. Hitung Metrik
report_lstm_ft = classification_report(y_test_lstm, y_pred_lstm_ft, # y_test_lstm dari Cell 9
                                      target_names=['Negative (0)', 'Positive (1)'],
                                      output_dict=True, zero_division=0)

print("\n--- Hasil Evaluasi Test Set LSTM + FastText ---")
print(classification_report(y_test_lstm, y_pred_lstm_ft,
                            target_names=['Negative (0)', 'Positive (1)'],
                            zero_division=0))

# Ekstrak F1 Macro dan Accuracy untuk perbandingan
acc_lstm_ft = report_lstm_ft['accuracy']
f1_lstm_ft = report_lstm_ft['macro avg']['f1-score']

Memulai evaluasi final LSTM + FastText pada Test Set...

--- Hasil Evaluasi Test Set LSTM + FastText ---
              precision    recall  f1-score   support

Negative (0)       0.85      0.94      0.89      1025
Positive (1)       0.93      0.83      0.87       975

    accuracy                           0.88      2000
   macro avg       0.89      0.88      0.88      2000
weighted avg       0.89      0.88      0.88      2000



# Perbandingan Hasil

In [ ]:
# Cell 12 (BARU): Perbandingan Akhir (3 Model)

# Impor fungsi untuk "menggambar" Markdown
from IPython.display import display, Markdown

# 1. Ambil hasil Transformer dari Cell 8
acc_trans = test_results_trans.get('eval_accuracy', 0)
f1_trans = test_results_trans.get('eval_f1', 0)

# 2. Ambil hasil LSTM Biasa dari Cell 11
# (Variabel acc_lstm dan f1_lstm sudah ada)

# 3. Ambil hasil LSTM + FastText dari Cell 11.D
# (Variabel acc_lstm_ft dan f1_lstm_ft sudah ada)

# --- INI ADALAH PERBAIKANNYA ---

# Buat string tabel dalam format Markdown
tabel_md = f"""
--- Hasil Perbandingan Klasifikasi Sentimen ---
Dataset: MyIM3 Reviews ({len(df_test)} sampel test)

| Model | Accuracy (Test) | F1 Score (Macro) |
| :--- | :---: | :---: |
| **Transformer (IndoBERT)** | **{acc_trans:.4f}** | **{f1_trans:.4f}** |
| **LSTM + FastText (cc.id)** | **{acc_lstm_ft:.4f}** | **{f1_lstm_ft:.4f}** |
| **LSTM (From Scratch)** | {acc_lstm:.4f} | {f1_lstm:.4f} |

---

"""

# Gunakan display(Markdown(...)) untuk mencetak tabel yang rapi
display(Markdown(tabel_md))


--- Hasil Perbandingan Klasifikasi Sentimen ---
Dataset: MyIM3 Reviews (2000 sampel test)

| Model | Accuracy (Test) | F1 Score (Macro) |
| :--- | :---: | :---: |
| **Transformer (IndoBERT)** | **0.8910** | **0.8909** |
| **LSTM + FastText (cc.id)** | **0.8835** | **0.8828** |
| **LSTM (From Scratch)** | 0.8720 | 0.8719 |

---

